# MaxCut

The cost function for the MaxCut problem as defined in the [original QAOA paper](https://arxiv.org/abs/1411.4028) is

$$
    \hat C = \frac 12 \sum_{(i, j) \in E(G)} (1 - \hat Z_i \hat Z_j),
$$

where $E(G)$ is the set of edges of the graph $G$.

In [ ]:
using DrWatson
@quickactivate "QAOA.jl"

include("../src/QAOA.jl")

using PyPlot
PyPlot.plt.style.use("paper.mplstyle")
using PyCall
np = pyimport("numpy")
nx = pyimport("networkx");

In [ ]:
N = 4
graph = nx.cycle_graph(N) 

figure(figsize=(3, 2))
nx.draw(graph, with_labels=true)

Note that we have to __shift the edges by 1__ when going from Python to Julia:

In [ ]:
h = zeros(N)
J = zeros(N, N)
for edge in graph.edges
    J[(edge .+ (1, 1))...] = -1/2.
end
J

In [ ]:
p = 1
problem = QAOA.Problem(p, h, J)

__Using the wrapper function:__

In [ ]:
max_cut_problem = QAOA.max_cut(graph, num_layers=p)

__Gradient optimization with [Zygote](https://fluxml.ai/Zygote.jl/latest/):__

In [ ]:
learning_rate = 0.01
cost, params, probs = QAOA.optimize_parameters(max_cut_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]); learning_rate=learning_rate)

__Optimization with [NLopt](https://nlopt.readthedocs.io/en/latest/):__

In [ ]:
cost, params, probs = QAOA.optimize_parameters(max_cut_problem, vcat([0.5 for _ in 1:p], [0.5 for _ in 1:p]), :LN_COBYLA)

In [ ]:
xlabels = []
for bstr in digits.(0:2^N-1, base=2, pad=N)
    push!(xlabels, "\$|" * prod([string(b) for b in bstr]) * "\\rangle\$")
end

figure(figsize=(5, 3.2))
ax = subplot(111)
bar(0:2^N-1, probs)
ax.set_xticks(0:2^N-1)
ax.set_xticklabels(xlabels, rotation=90)
minorticks_off()
tight_layout()

The states `5` $\equiv |1010\rangle$ and `10` $\equiv |0101\rangle$ are indeed the correct solutions! Observe that `Yao.jl` uses the _opposite_ order when decoding binaries compared to, e.g., Pennylane (i.e. our solution `5` is their solution `10`).